<a href="https://colab.research.google.com/github/KunalParkhade/product-tagging-cnn/blob/main/Product_Tagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torchmetrics
!pip install torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.2/866.2 kB 32.5 MB/s eta 0:00:00


In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy, Precision, Recall

In [3]:
# Load datasets
from torchvision import datasets
import torchvision.transforms as transforms

train_data = datasets.FashionMNIST(root='./content/drive/MyDrive/Datasets/FashionMNIST', train=True, download=True,
                                   transform = transforms.ToTensor())
test_data = datasets.FashionMNIST(root='./content/drive/MyDrive/Datasets/FashionMNIST', train=False, download=True,
                                   transform = transforms.ToTensor())

100%|██████████| 26421880/26421880 [00:01<00:00, 17612279.96it/s]


Extracting ./content/drive/MyDrive/Datasets/FashionMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./content/drive/MyDrive/Datasets/FashionMNIST/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 343834.56it/s]


Extracting ./content/drive/MyDrive/Datasets/FashionMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./content/drive/MyDrive/Datasets/FashionMNIST/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 6243628.12it/s]


Extracting ./content/drive/MyDrive/Datasets/FashionMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./content/drive/MyDrive/Datasets/FashionMNIST/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 15216544.74it/s]

Extracting ./content/drive/MyDrive/Datasets/FashionMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./content/drive/MyDrive/Datasets/FashionMNIST/FashionMNIST/raw



# 1. Define the CNN

In [4]:
class GarmentClassifier(nn.Module):
  def __init__(self):
    super(GarmentClassifier, self).__init__()
    # Convolutional Layer 1
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
    # ReLU activation function
    self.relu1 = nn.ReLU()
    # Pooling layer 1
    self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

    # Convolutional layer 2
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
    # ReLU activation function
    self.relu2 = nn.ReLU()
    # Pooling layer 2
    self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

    # Fully connected layer
    self.fc1 = nn.Linear(64 * 7 * 7, 1000)
    # ReLU activation function
    self.relu3 = nn.ReLU()

    # Fully connected layer 2 (Output layer)
    self.fc2 = nn.Linear(1000, 10)    # 10 classes for FashionMNIST


  def forward(self, x):
    # Apply convolution 1, ReLU, and pooling 1
    x = self.pool1(self.relu1(self.conv1(x)))
    # Apply convolution 2, ReLU, and pooling 2
    x = self.pool2(self.relu2(self.conv2(x)))
    # Flatten the output
    x = x.view(-1, 64 * 7 * 7)
    # Apply fully connected layer 1 and ReLU
    x = self.relu3(self.fc1(x))
    # Apply fully connected layer 2 and ReLU
    x = self.fc2(x)
    return x

https://chatgpt.com/share/6168aacb-45a0-4bfc-acf4-8293dc45f2dd

# 2. Training the CNN

Now, let's define the training loop:

In [5]:
# Initialize the model, loss function and optimizer
model = GarmentClassifier()
criterion = nn.CrossEntropyLoss() # Suitable loss for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# DataLoader for batching
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# Training loop
epochs = 2    # We limit epochs to 2 to keep runtime down
for epoch in range(epochs):
  model.train()
  running_loss = 0.0
  for images, labels in train_loader:
    optimizer.zero_grad()   # Zero the gradients
    outputs = model(images)   # Forward pass
    loss = criterion(outputs, labels)   # Calculate loss
    loss.backward()   # Backward pass
    optimizer.step()    # Update weights

    running_loss += loss.item()

  print(f'Epoch {epoch+1}, Loss:{running_loss/len(train_loader)}')

Epoch 1, Loss:0.4139006641278389
Epoch 2, Loss:0.26094082121782974


# 3. Testing the CNN
Finally, let's test the model on the test dataset and store the predictions:

In [6]:
# Testing loop
model.eval()
predictions = []
correct = 0
total = 0

with torch.no_grad():
  for images, labels in test_loader:
    outputs = model(images)   # Forward pass
    _, predicted = torch.max(outputs, 1)    # Get the index of the max log-probability
    predictions.extend(predicted.cpu().numpy())
    total += labels.size(0)
    correct += (predicted==labels).sum().item()


accuracy = correct / total
print(f'Accuracy: {accuracy*100: .2f}%')

Accuracy:  90.13%
